In [15]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.prompts import PromptTemplate, format_document
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from langchain_qdrant import QdrantVectorStore

import pandas as pd


In [16]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGUAGE_MODEL = "gpt-3.5-turbo-instruct"
model = ChatOpenAI()

In [19]:
template: str = """/
    imagine that you are a clerk at a supermarket store. Consider the following description for a product that the customer is looking for: /
      {description} /
    use the below retrived products information and choose the most relavent one for the needs of the customer: /
      {context} /
      give as response, the information on the product most relavent to the need of the customer. You should strictly use one of the products above. /
      In your response include all the relavent information on the product.
    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["description", "context"],
    template="{description}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)


# Create a template including your metadata
doc_template = PromptTemplate.from_template("""
Product: {product}
Category: {category} → {sub_category}
Brand: {brand}
Sale Price: ₹{sale_price}, Market Price: ₹{market_price}
Type: {type}, Rating: {rating}
Description:
{page_content}
""".strip())


def get_retriever(query: str):
    qdrant_url = os.getenv("QDRANT_URL")
    qdrant_api_key = os.getenv("QDRANT_API_KEY")
    collection_name = "product_catalog"

    embedding_model = OpenAIEmbeddings()

    vectorstore = QdrantVectorStore.from_existing_collection(
        collection_name=collection_name,
        embedding=embedding_model,          
        url=qdrant_url,
        api_key=qdrant_api_key,
        prefer_grpc=False,
        timeout=60,
    )
    return vectorstore.as_retriever(search_kwargs={"k": 3})



def format_docs(docs):
    return "\n---\n".join(format_document(doc, doc_template) for doc in docs)


def generate_response(retriever, query: str) -> str:
    # Step 1: Retrieve documents (with metadata)
    docs = retriever.invoke(query)

    # Step 2: Format documents for the prompt
    context = format_docs(docs)

    # Step 3: Create the full chain (prompt → model → parser)
    chain = (
        {"context": RunnablePassthrough(), "description": RunnablePassthrough()}
        | chat_prompt_template  # assumes you've already defined this
        | model                 # assumes you've defined your ChatOpenAI model
        | StrOutputParser()
    )

    # Step 4: Invoke the chain
    return chain.invoke({"description": query, "context": context})


In [ ]:
user_query = "I want to cook a persian food for tree person and with max 300 rubles price."
retriever = get_retriever(user_query)

response = generate_response(retriever, user_query)
print(f"Response: {response}")

Response: Based on your request to cook a Persian food for three people within a budget of max 300 rubles, the most relevant product from the information provided would be the **"Hara Bhara Veg Kebab"** by **Safal**. Here are the details of the product:

- **Category:** Snacks & Branded Foods → Frozen Veggies & Snacks
- **Brand:** Safal
- **Sale Price:** ₹70.0
- **Market Price:** ₹70.0
- **Type:** Frozen Veg Snacks
- **Rating:** 3.9
- **Description:** The Goodness of green loaded with taste! It is rich in green peas, containing 50% peas, spinach, spices, and condiments. A flavorful, traditional recipe ready in 3 minutes. Enjoy the delicious kebab with family & friends.

This product fits within your budget and can be a tasty addition to your Persian meal for three people.
